# Inverse problem for the wave equation using an operator recurrent neural network

We consider the inverse problem to find $a$ in the below wave equation given 
the Neumann-to-Dirichlet map 

$$
\Lambda h = u|_{x = 0},
$$

where $u$ is the solution to the problem 

$$
\begin{cases}
\partial_t^2 u - a(x) \partial_x^2 u = 0, & \text{on $(0,T) \times (0,L)$},
\\
\partial_x u|_{x=0} = h, \quad \partial_x u|_{x=L} = 0,
\\
u|_{t=0} = 0, \quad \partial_t u|_{t=0} = 0.
\end{cases}
$$

Here we consider only a subproblem related to the inverse problem to find $a$.
In Section 2 of 

> Jussi Korpela, Matti Lassas and Lauri Oksanen.
> _Regularization strategy for an inverse problem for a 1 + 1 dimensional wave equation_.
> Inverse Problems 32, 065001, 2016.
> <https://doi.org/10.1088/0266-5611/32/6/065001> 

it was shown that $\Lambda$ determines the following volumes 

$$
V(r) = \int_0^{\chi(r)} \frac{1}{c(x)^2} dx
$$

and that these volumes then determine $a$.
Here $c^2 = a$ and $\chi$ is the inverse function of $\tau$ defined by

$$
\tau(y) = \int_0^y \frac{1}{c(x)} dx.
$$


We consider the subproblem to compute a single volume $V(r_0)$, with fixed $r_0>0$, given $\Lambda$. 
We solve this problem using a neural network, with the network architecture taken from 

> Maarten V. de Hoop, Matti Lassas, Christopher A. Wong. _Deep learning architectures for nonlinear operator functions and nonlinear inverse problems_. [arXiv:1912.11090](https://arxiv.org/abs/1912.11090)

The training data consists of pairs $(\Lambda, V(r_0))$ corresponding to different functions $a$.
Here $\Lambda$ is, of course, discretized, and the details of the discretization are discussed in the notebook describing the generation of the data. 


# Initialization

In [89]:
import numpy as np
import torch

import opnet
from volume_inversion_data import generate_data, save_data, load_data

PATH = './volume_inversion_NOReLU.pth'
PATH = './volume_inversion_ReLU.pth'

Specify the network model and the loss function.

In [90]:

dim = 126 # this needs be the size of Lambda_h
num_layers = 5
# luodaan uusi neuroverkko
# model = opnet.OperatorNet(dim, 2*num_layers, scalar_output=True, useReLU=False)
model = opnet.OperatorNet(dim, num_layers, scalar_output=True, useReLU=True)
loss_fn = torch.nn.MSELoss()

# Generation of training data

In [91]:
save_data(*generate_data(60), "volume_inversion_train_data.npz")
save_data(*generate_data(10), "volume_inversion_test_data.npz")

# Training

In [92]:
import wave_training_and_testing
import os.path

# update changes
from importlib import reload 
reload(wave_training_and_testing)
reload(opnet)

lr=1e-2

# upload the neural network used earlier:
# if os.path.exists(PATH):
#     # PATH-tiedostoon on tallennettu verkko ja tämä lataa sen kertoimet ylempänä 
#     # luotuun verkkoon, eli jatketaan vanhalla verkolla
#     model.load_state_dict(torch.load(PATH)) 
# else:
#     print("no old PATH, I'll make a new one")

wave_training_and_testing.wave_training_and_testing(model, loss_fn, lr)


1      nan 

2      nan 

3      nan 

4      nan 

5      nan 



KeyboardInterrupt: 

Choose the optimization method.

In [ ]:
# Learning rate parameter is from the quickstart guide 
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Loop over the training data multiple times (epochs) and 
save the optimized parameters. 

In [ ]:
# for epoch in range(2): 
#     print(f"Epoch {epoch+1}\n-------------------------------")
#     for batch, (X, y) in enumerate(train_loader):
#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # Print statistics
#         if batch % 10 == 0:
#             n, N = (batch + 1) * len(X), len(train_loader.dataset)
#             print(f"loss: {loss.item():>7f}  [{n:>5d}/{N:>5d}]")

torch.save(model.state_dict(), PATH)

# Testing

If we have already trained the network, we can just load its parameters. (Note that we still need to run the initialization.)

In [ ]:
## Load trained variables
# model.load_state_dict(torch.load(PATH))

In [ ]:
# # Load the testing data
# test_loader = torch.utils.data.DataLoader(
#     load_data("volume_inversion_test_data.npz"), 
#     batch_size=64)

Compute a couple of samples.

In [ ]:
# dataiter = iter(test_loader)
# X, y = dataiter.next()
# with torch.no_grad():
#     pred = model(X)
# print("True: ")
# print(y[:2])
# print("Prediction: ")
# print(pred[:2])

In [ ]:
# num_batches = len(test_loader)
# test_loss = 0
# with torch.no_grad():
#     for X, y in test_loader:
#         pred = model(X)
#         test_loss += loss_fn(pred, y).item()
# test_loss /= num_batches
# print(f"Avg loss: {test_loss:>8f} \n")